# Customer Support Bot Project Solution

This notebook contains the complete solution for the Customer Support Bot project.

## Section 1: Setup

First, let's install the Aurite package and configure our environment.

In [27]:
# Install the Aurite package
%pip install aurite==0.3.18


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
# Configure API Keys
import os
from getpass import getpass

try:
  from google.colab import userdata #type: ignore
  os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
  os.environ["SMITHERY_API_KEY"] = userdata.get('SMITHERY_API_KEY')
  os.environ["SMITHERY_PROFILE_ID"] = userdata.get('SMITHERY_PROFILE_ID')
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print(f"Error fetching from Colab's secret manager: {e}")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
if "SMITHERY_API_KEY" not in os.environ:
    os.environ["SMITHERY_API_KEY"] = getpass("Enter your Smithery API key: ")
if "SMITHERY_PROFILE_ID" not in os.environ:
    os.environ["SMITHERY_PROFILE_ID"] = getpass("Enter your Smithery Profile ID: ")

In [29]:
# Helper function for displaying agent responses
from IPython.display import display, Markdown

def display_agent_response(agent_name: str, query: str, response: str):
  """Formats and displays the agent's response in a structured Markdown block."""
  output = f"""
  <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
    <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
      <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
        <span style=\"margin-right: 8px;\">🤖</span>
        Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
      </h3>
    </div>
    <div style=\"padding: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Your Query:</strong>
      </p>
      <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
        <em>\\\"{query}\\\"</em>
      </p>
      <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
      <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
        <strong>Result:</strong>
      </p>
      <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
        {response}
      </div>
    </div>
  </div>
  """
  display(Markdown(output))

In [30]:
# Set up a dedicated directory for our knowledge base files
# and configure the file storage server to use it.
import os
from pathlib import Path

# Create a dedicated directory for the knowledge base
docs_dir = Path("customer_support_docs")
docs_dir.mkdir(exist_ok=True)

# Set the environment variable to this new directory.
# The local_file_storage server will use this path as its root.
os.environ['AURITE_PROJECT_DIR'] = str(docs_dir.resolve())

print(f"✅ Set AURITE_PROJECT_DIR to: {os.environ['AURITE_PROJECT_DIR']}")

✅ Set AURITE_PROJECT_DIR to: /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/customer_support_docs


In [31]:
# Initialize Aurite
from aurite import Aurite

aurite = Aurite()
await aurite.initialize()
print("✅ Aurite initialized successfully!")

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


✅ Aurite initialized successfully!


## Section 2: Create the GadgetCo Knowledge Base

In [32]:
# Create the FAQ document
faq_content = """# GadgetCo Smart-Widget FAQ

## General Questions
**Q: What is the Smart-Widget?**
A: The Smart-Widget is GadgetCo's flagship IoT device that combines home automation, health monitoring, and entertainment features in one sleek package.
"""
faq_path = docs_dir / "faq.md"
with open(faq_path, "w") as f:
    f.write(faq_content)
print(f"✅ Created {faq_path}")

✅ Created customer_support_docs/faq.md


In [33]:
# Create the manuals directory and Smart-Widget manual
manuals_dir = docs_dir / "manuals"
manuals_dir.mkdir(exist_ok=True)
manual_content = """# Smart-Widget User Manual

## Technical Specifications
- Processor: ARM Cortex-A53 quad-core
- RAM: 1GB DDR3
"""
manual_path = manuals_dir / "smart_widget_manual.txt"
with open(manual_path, "w") as f:
    f.write(manual_content)
print(f"✅ Created {manual_path}")

✅ Created customer_support_docs/manuals/smart_widget_manual.txt


## Section 3: Understanding the Tools You'll Need

In [34]:
# Let's register a simple agent to test the local file storage tools
from aurite.config.config_models import AgentConfig

test_agent_config = AgentConfig(
    name="File Test Agent",
    system_prompt="You are a helpful assistant that can read and list files. Use the available tools to help users understand what files are available and read their contents.",
    mcp_servers=["local_file_storage"]
)

await aurite.register_agent(test_agent_config)
print("✅ Registered File Test Agent")

INFO     [aurite.host.host] Attempting to dynamically register client: local_file_storage
INFO     [aurite.host.host] Client 'local_file_storage' dynamically registered and initialized successfully.


✅ Registered File Test Agent


In [35]:
# Test the file listing capability
query = "Can you list what files are available in this directory?"

result = await aurite.run_agent(
    agent_name="File Test Agent",
    user_message=query
)

display_agent_response("File Test Agent", query, result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'File Test Agent' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'File Test Agent'...
INFO     [aurite.host.host] Executing tool 'list_directory' on client 'local_file_storage' for agent 'File Test Agent'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'File Test Agent' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">File Test Agent</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"Can you list what files are available in this directory?\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The following files are available in this directory:
1. faq.md
2. manuals/smart_widget_manual.txt
      </div>
    </div>
  </div>
  

## Section 4: Your Challenge - Build Chip the Customer Support Agent

In [36]:
# Create and register Chip the customer support agent
chip_config = AgentConfig(
    name="Chip",
    include_history=True, # IMPORTANT: Enable conversation history
    system_prompt="""You are Chip, a friendly, patient, and professional customer support representative for GadgetCo. Your primary goal is to help users with their Smart-Widget device by providing accurate information from the official company knowledge base.\n\n**Your Process:**\nYou MUST follow this process strictly in order:\n1.  **Check the FAQ**: First, use the `read_file` tool to check `faq.md`. This file contains answers to common questions.\n2.  **Check the Manual**: If the answer is not in the FAQ, use the `read_file` tool to check the `manuals/smart_widget_manual.txt` file. This file contains detailed technical specifications and troubleshooting steps.\n3.  **Use External Search**: Only if you cannot find the answer in either the FAQ or the manual, you may use the `duckduckgo_search` tool to search the web for more information.\n4.  **Escalate if Necessary**: If you cannot find a definitive answer after checking all sources, you MUST inform the user that you don't have the information and that you will escalate their question to a human support agent.\n\n**Your Rules:**\n- **NEVER GUESS**: If you don't know the answer, say so. Do not make up information.\n- **CITE YOUR SOURCES**: When you provide an answer, you MUST state where you found the information (e.g., \"I found that in the FAQ,\" or \"According to the user manual...\").\n- **BE HELPFUL AND HONEST**: Maintain a friendly and professional tone at all times.""",
    mcp_servers=["local_file_storage", "duckduckgo_search"]
)

await aurite.register_agent(chip_config)

print("✅ Chip the customer support agent is registered and ready!")

INFO     [aurite.host.host] Attempting to dynamically register client: duckduckgo_search
INFO     [aurite.host.host] Client 'duckduckgo_search' dynamically registered and initialized successfully.


✅ Chip the customer support agent is registered and ready!


## Section 5: Test Your Agent

In [37]:
# Test 1: Simple FAQ question
query = "What's the warranty period for the Smart-Widget?"

result = await aurite.run_agent(
    agent_name="Chip",
    user_message=query
)
display_agent_response("Chip", query, result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'Chip' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Chip'...
INFO     [aurite.host.host] Executing tool 'read_file' on client 'local_file_storage' for agent 'Chip'
INFO     [aurite.host.host] Executing tool 'read_file' on client 'local_file_storage' for agent 'Chip'
INFO     [aurite.host.host] Executing tool 'search' on client 'duckduckgo_search' for agent 'Chip'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Chip' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Chip</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"What's the warranty period for the Smart-Widget?\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The warranty period for the Smart-Widget is a 2-year limited warranty that covers manufacturing defects under normal use and installation conditions. It does not cover damage due to misuse, neglect, improper installation, or unauthorized modifications. To file a warranty claim, you can contact Swidget Corp. at support@swidget.com with the subject line "Warranty Claim" (source: [Understanding Your Swidget Warranty - Swidget](https://support.swidget.com/hc/en-us/articles/38062624613012-Understanding-Your-Swidget-Warranty)).
      </div>
    </div>
  </div>
  

In [38]:
# Test 2: Technical question requiring manual lookup
query = "What are the technical specifications of the Smart-Widget? I need to know the processor and RAM details."

result = await aurite.run_agent(
    agent_name="Chip",
    user_message=query
)
display_agent_response("Chip", query, result.primary_text)

WARNING  [aurite.execution.facade] Facade: Agent 'Chip' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Chip'...
INFO     [aurite.host.host] Executing tool 'read_file' on client 'local_file_storage' for agent 'Chip'
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Chip' conversation finished.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Chip</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"What are the technical specifications of the Smart-Widget? I need to know the processor and RAM details.\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        The technical specifications of the Smart-Widget are as follows:
- Processor: ARM Cortex-A53 quad-core
- RAM: 1GB DDR3

I found this information in the Smart-Widget User Manual.
      </div>
    </div>
  </div>
  

## Section 7: Chat with Chip!

Now that Chip is built and tested, you can have a conversation with it. Run the cell below to start an interactive chat session. Type "quit" when you're finished.

In [39]:
import uuid
import asyncio

# Create a unique session ID for this conversation
session_id = str(uuid.uuid4())

# Clear any previous in-memory history for this session ID before starting
# This ensures each run of the cell is a fresh conversation
if aurite.execution:
    aurite.execution.clear_history_cache(session_id)

print(f"Starting new chat session: {session_id}")
print('---')
print('Hello! I am Chip, your GadgetCo support assistant. How can I help you today?')
print('Type your questions below, or type "quit" to exit.')
print('---')

async def chat_loop():
    while True:
        try:
            # Get user input from a separate thread to avoid blocking the event loop
            query = await asyncio.to_thread(input, "You: ")

            # Check for exit conditions: "quit" or an empty string from pressing Escape
            if query.lower() == 'quit' or query == "":
                print("\n🤖 Chip: Goodbye! Thanks for chatting.")
                break

            # Run the agent with the user's query and the session ID
            result = await aurite.run_agent(
                agent_name="Chip",
                user_message=query,
                session_id=session_id
            )

            # Display the agent's response using the rich display helper function
            display_agent_response("Chip", query, result.primary_text)

            # A small delay to ensure the rich display renders before the next input prompt
            await asyncio.sleep(0.1)

        except (KeyboardInterrupt, EOFError):
            print("\n🤖 Chip: Chat session ended.")
            break

# Run the asynchronous chat loop
await chat_loop()


Starting new chat session: a136538c-1369-4a63-b115-e8c3569b3cdd
---
Hello! I am Chip, your GadgetCo support assistant. How can I help you today?
Type your questions below, or type "quit" to exit.
---


WARNING  [aurite.execution.facade] Facade: Agent 'Chip' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Chip'...
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Chip' conversation finished.
INFO     [aurite.execution.facade] Facade: Saved 2 history turns to in-memory cache for session 'a136538c-1369-4a63-b115-e8c3569b3cdd'.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Chip</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"Hello Chip! How are you today?\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        Hello! I'm here and ready to assist you with any questions or issues you have regarding your Smart-Widget device. How can I help you today?
      </div>
    </div>
  </div>
  

WARNING  [aurite.execution.facade] Facade: Agent 'Chip' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-3.5-turbo using direct API calls.
INFO     [aurite.execution.facade] Facade: Loaded 2 history turns from in-memory cache for session 'a136538c-1369-4a63-b115-e8c3569b3cdd'.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Chip'...
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Chip' conversation finished.
INFO     [aurite.execution.facade] Facade: Saved 4 history turns to in-memory cache for session 'a136538c-1369-4a63-b115-e8c3569b3cdd'.



  <div style="border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
    <div style="background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;">
      <h3 style="margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;">
        <span style="margin-right: 8px;">🤖</span>
        Agent Response: <code style="background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;">Chip</code>
      </h3>
    </div>
    <div style="padding: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Your Query:</strong>
      </p>
      <p style="background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;">
        <em>\"What did I say in my last message? Just checking if you have conversation history persistence.\"</em>
      </p>
      <hr style="border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;">
      <p style="margin: 0 0 10px 0; color: #6B7280; font-size: 14px;">
        <strong>Result:</strong>
      </p>
      <div style="background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;">
        In your last message, you asked me, "How are you today?" Is there anything specific you would like to know or discuss about your Smart-Widget device?
      </div>
    </div>
  </div>
  


🤖 Chip: Goodbye! Thanks for chatting.
